In [ ]:
!pip install -q sklearn

# New Section

In [ ]:
%tensorflow_version 2.x

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

We convert our data into feature_columns. TensorFlow feature column is something which we will feed to our linear estimator.
To convert our raw data into feature columns, we use tf.feature_column function.

Think of feature columns as the intermediaries between raw data and Estimators. Feature columns are very rich, enabling you to transform a diverse range of raw data into formats that Estimators can use, allowing easy experimentation.
In simple words feature column are bridge between raw data and estimator or model.


In [ ]:
dftrain = pd.read_csv("/content/diabetes_training.csv") # This will be our training data. 2/3rd of the total data
dfeval = pd.read_csv('/content/diabetes_testing.csv') # This will be our testing data. 1/3rd of the total data

In [ ]:
dftrain

In [ ]:
dftrain = pd.read_csv("/content/diabetes_training.csv") # This will be our training data. 2/3rd of the total data
dfeval = pd.read_csv('/content/diabetes_testing.csv') # This will be our testing data. 1/3rd of the total data
y_train = dftrain.pop('Outcome')
y_eval = dfeval.pop('Outcome')

NUMERIC_COLUMN = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]

feature_columns = []

for feature_name in NUMERIC_COLUMN:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

def make_input_fn(data_df, label_df, num_epochs=30, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn)  # train
result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on tetsing data

clear_output()  # clears console output
print(result['accuracy'])  # the result variable is simply a dict of stats about our model


NameError: ignored

In [ ]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[2]) # For a particular record
print(f"Probability of having the diabetes: {result[2]['probabilities'][1]}") # Probability of a particular record
if result[2]['probabilities'][1] > 0.50:
  print("The person has the diabetes")
else:
  print("The person does not have the diabetes")

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/canned/linear.py:1478: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  getter=tf.compat.v1.get_variable)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpopu3u4ci/model.ckpt-390
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Pregnancies                  3.000
Glucose                     99.000
BloodPressure               54.000
SkinThickness               19.000
Insulin                     86.000
BMI                         25.600
DiabetesPedigreeFunction     0.154
Age                         24.000
Name: 2, dtype: float64
Probability of having the diabetes: 0.18529997766017914
The person does not have the diabetes


In [ ]:
dftrain.Age.hist(bins=20)   # Graph of age vs no. of women. Most of the women are in their 20s or 30s


In [ ]:
dftrain.BMI.hist(bins=20)